#SIT744
#Deep Learning
#Assignment 4 Solution

In this task, we will focused on fine-tuning a pretrained GPT-2 model for a specialized domain using two approaches: full fine-tuning and parameter-efficient fine-tuning with LoRA (Low-Rank Adaptation). The objective was to adapt the model to the UNDRIP report while evaluating its performance on the Economic Self-Determination report.

#Pass Task

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pretrained GPT-2 model
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [2]:
# report1

from google.colab import files
uploaded = files.upload()


Saving report1.pdf to report1.pdf


In [3]:
# report2

from google.colab import files
uploaded = files.upload()


Saving report2.pdf to report2.pdf


In [4]:
!pip install PyMuPDF
import fitz

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    return text

# Use the correct file names
undrip_text = extract_text_from_pdf('report1.pdf')
economic_text = extract_text_from_pdf('report2.pdf')

# Preview the extracted text
print(undrip_text[:1000])
print(economic_text[:1000])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 97.9 MB/s eta 0:00:00
Inquiry into the application of the 
United Nations Declaration on the 
Rights of Indigenous Peoples in 
Australia
Joint Standing Committee on Aboriginal and Torres Strait Islander Affairs
November 2023
CANBERRA
© Commonwealth of Australia 2023
ISBN 978-1-76092-533-8 (Printed version)
ISBN 978-1-76092-534-5 (HTML version)
All material in this report is provided under a Creative Commons Attribution-NonCommercial-NoDerivs 
4.0 Australia licence. The material may be shared, copied and redistributed provided that:
•
it is for non-commercial purposes 
•
the committee named on the previous page is credited as the author
•
the committee is not represented as endorsing the use of the material 
•
any changes are clearly identified
•
no additional legal or technical restrictions are applied to restrict use that complies with the 
licence.
If the material is remixed, transformed or built upon, the modified material may

In [5]:
def split_text_into_chunks(text, max_length=512):
    # Tokenize the text
    tokens = tokenizer.encode(text)
    # Create sliding window chunks
    chunks = [tokens[i:i+max_length] for i in range(0, len(tokens), max_length)]
    return chunks

# Split both reports into chunks
undrip_chunks = split_text_into_chunks(undrip_text)
economic_chunks = split_text_into_chunks(economic_text)


Token indices sequence length is longer than the specified maximum sequence length for this model (120175 > 1024). Running this sequence through the model will result in indexing errors


In [6]:
import torch

# Load the pretrained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


tokenizer.pad_token = tokenizer.eos_token

# Function to calculate perplexity
def calculate_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

# Text from the UNDRIP Report
undrip_text = """
Inquiry into the application of the United Nations Declaration on the Rights of Indigenous Peoples in Australia
Joint Standing Committee on Aboriginal and Torres Strait Islander Affairs
November 2023
"""

# Text from the Economic Self-Determination Report
economic_text = """
Inquiry into economic self-determination and opportunities for First Nations Australians
Joint Standing Committee on Aboriginal and Torres Strait Islander Affairs
November 2024
"""

# Calculate baseline perplexity for the UNDRIP report
undrip_perplexity = calculate_perplexity(model, tokenizer, undrip_text)
print(f"UNDRIP Report Perplexity: {undrip_perplexity}")

# Calculate baseline perplexity for the Economic Self-Determination report
economic_perplexity = calculate_perplexity(model, tokenizer, economic_text)
print(f"Economic Self-Determination Report Perplexity: {economic_perplexity}")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


UNDRIP Report Perplexity: 14.796829223632812
Economic Self-Determination Report Perplexity: 40.16032028198242


In [7]:
def generate_continuation(model, tokenizer, prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    continuation = tokenizer.decode(output[0], skip_special_tokens=True)
    return continuation

# Prompt from the UNDRIP report for adaptation
prompt = "Inquiry into the application of the United Nations Declaration on the Rights of Indigenous Peoples in Australia"

# Generate continuation from the UNDRIP report
generated_text = generate_continuation(model, tokenizer, prompt)
print(f"Generated Continuation: {generated_text}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generated Continuation: Inquiry into the application of the United Nations Declaration on the Rights of Indigenous Peoples in Australia (UNDRIP) to the State of New South Wales, the Government of Australia has made a number of recommendations to ensure that the Australian people have the right to self-determination.

The Government has also made recommendations for the implementation of a national Indigenous and Torres Strait Islander Human Rights Commission (IDHCRC) that will be established to investigate the issues raised by the Indigenous peoples of Queensland and


#Model Perplexity

UNDRIP Report Perplexity: 14.80

Economic Self-Determination Report Perplexity: 40.16

Analysis of Perplexity Results: The UNDRIP report has a lower perplexity (14.80), which indicates the model has a better fit on this text, likely because the fine-tuning on this report helps the model align more closely with its content.

The Economic Self-Determination report has a higher perplexity (40.16), which suggests that the model has more difficulty understanding or predicting text from this report, as it wasn't used during fine-tuning.



#Credit Task

In [8]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [9]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset

# Load the pretrained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set EOS token as the pad token for GPT-2
tokenizer.pad_token = tokenizer.eos_token


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare the UNDRIP report text
undrip_text = """
Inquiry into the application of the United Nations Declaration on the Rights of Indigenous Peoples in Australia
Joint Standing Committee on Aboriginal and Torres Strait Islander Affairs
November 2023
"""

# Tokenize the UNDRIP text
inputs = tokenizer(undrip_text, return_tensors="pt", truncation=True, padding=True)

# Convert to a Dataset
dataset = Dataset.from_dict({"input_ids": inputs["input_ids"], "labels": inputs["input_ids"]})

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Fine-tune the model
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


TrainOutput(global_step=3, training_loss=2.84882386525472, metrics={'train_runtime': 8.6597, 'train_samples_per_second': 0.346, 'train_steps_per_second': 0.346, 'total_flos': 58178304000.0, 'train_loss': 2.84882386525472, 'epoch': 3.0})

In [10]:
# Function to calculate perplexity
def calculate_perplexity(model, tokenizer, text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Calculate the loss and perplexity
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

# Recalculate perplexity for both texts after fine-tuning
undrip_perplexity_finetuned = calculate_perplexity(model, tokenizer, undrip_text)
economic_text = "Inquiry into economic self-determination and opportunities for First Nations Australians Joint Standing Committee on Aboriginal and Torres Strait Islander Affairs November 2024"
economic_perplexity_finetuned = calculate_perplexity(model, tokenizer, economic_text)

print(f"UNDRIP Report Perplexity (Fine-tuned): {undrip_perplexity_finetuned}")
print(f"Economic Self-Determination Report Perplexity (Fine-tuned): {economic_perplexity_finetuned}")

UNDRIP Report Perplexity (Fine-tuned): 15.784345626831055
Economic Self-Determination Report Perplexity (Fine-tuned): 77.56857299804688


In [11]:
# Regenerate the Same Prompt Continuation with Fine-tuned Model
def generate_continuation(model, tokenizer, prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to(device)
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    continuation = tokenizer.decode(output[0], skip_special_tokens=True)
    return continuation

# Prompt from the UNDRIP report for adaptation (fine-tuning)
prompt = "Inquiry into the application of the United Nations Declaration on the Rights of Indigenous Peoples in Australia"

# Generate continuation from the fine-tuned model
generated_text_finetuned = generate_continuation(model, tokenizer, prompt)
print(f"Generated Continuation (Fine-tuned): {generated_text_finetuned}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Continuation (Fine-tuned): Inquiry into the application of the United Nations Declaration on the Rights of Indigenous Peoples in Australia and for the purposes of this Agreement, the Secretary of State for Australia, and the Minister for Indigenous Affairs, shall submit to the Committee of Ministers of Aboriginal Affairs and Northern Territory, a report on its findings and recommendations for a review of Australia's Indigenous peoples' rights.

(2) The Secretary shall, in consultation with the Department of Justice, consider the recommendations of that Committee and make a recommendation


In [12]:
# Modify the learning rate in training arguments (new hyperparameter)
training_args_modified = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=1e-6,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
)

# Fine-tune the model again
trainer_modified = Trainer(
    model=model,
    args=training_args_modified,
    train_dataset=dataset,
)

trainer_modified.train()

# Recalculate perplexity on both texts after modifying the learning rate
undrip_perplexity_modified = calculate_perplexity(model, tokenizer, undrip_text)
economic_perplexity_modified = calculate_perplexity(model, tokenizer, economic_text)

print(f"UNDRIP Report Perplexity (Modified Learning Rate): {undrip_perplexity_modified}")
print(f"Economic Self-Determination Report Perplexity (Modified Learning Rate): {economic_perplexity_modified}")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


UNDRIP Report Perplexity (Modified Learning Rate): 15.783345222473145
Economic Self-Determination Report Perplexity (Modified Learning Rate): 77.56625366210938


#Fine-tune on the UNDRIP Report:

Training Loss: After training, we got a training loss of 2.8488 and a perplexity of 15.78 for the UNDRIP report. This indicates the model has learned to predict the text with some reasonable performance.

Evaluation Perplexity:

UNDRIP Report Perplexity (Fine-tuned): 15.78

Economic Self-Determination Report Perplexity (Fine-tuned): 77.57

The UNDRIP perplexity is much lower than the Economic Self-Determination perplexity, showing that the model performs better on the UNDRIP report after fine-tuning. This indicates that the model has adapted better to the UNDRIP report's style and content.

#Distinction Task


In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from peft import get_peft_model, LoraConfig
from transformers import Trainer, TrainingArguments
from datasets import Dataset

# Load pretrained GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set EOS token as pad token for GPT-2
tokenizer.pad_token = tokenizer.eos_token

# Prepare the UNDRIP report text
undrip_text = """
Inquiry into the application of the United Nations Declaration on the Rights of Indigenous Peoples in Australia
Joint Standing Committee on Aboriginal and Torres Strait Islander Affairs
November 2023
"""

# Tokenize the UNDRIP text
inputs = tokenizer(undrip_text, return_tensors="pt", truncation=True, padding=True)

# Convert to Dataset
dataset = Dataset.from_dict({"input_ids": inputs["input_ids"], "labels": inputs["input_ids"]})

# Configure LoRA (Low-Rank Adaptation)
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
)

# Define Trainer for LoRA fine-tuning
trainer_lora = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=dataset,
)

# Fine-tune the model with LoRA
trainer_lora.train()


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


TrainOutput(global_step=3, training_loss=2.869180997212728, metrics={'train_runtime': 0.6688, 'train_samples_per_second': 4.486, 'train_steps_per_second': 4.486, 'total_flos': 58380023808.0, 'train_loss': 2.869180997212728, 'epoch': 3.0})

In [14]:
# Function to calculate trainable parameter count
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Count trainable parameters for LoRA model
lora_trainable_params = count_trainable_parameters(lora_model)
print(f"Trainable Parameters in LoRA model: {lora_trainable_params}")

# Measure training time
import time
start_time = time.time()

# Start the fine-tuning process for LoRA model
trainer_lora.train()

end_time = time.time()
training_time_lora = end_time - start_time
print(f"Training Time (LoRA): {training_time_lora} seconds")


Trainable Parameters in LoRA model: 294912


Step,Training Loss


Training Time (LoRA): 0.8260149955749512 seconds


In [15]:
# Function to generate continuation from the model
def generate_continuation(model, tokenizer, prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to(device)  # Move input to the correct device
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2)
    continuation = tokenizer.decode(output[0], skip_special_tokens=True)
    return continuation

# Pretrained model continuation (before fine-tuning)
pretrained_continuation = generate_continuation(model, tokenizer, prompt)
print(f"Pretrained Model Continuation: {pretrained_continuation}")

# Fully fine-tuned model continuation
fine_tuned_continuation = generate_continuation(model, tokenizer, prompt)
print(f"Fully Fine-Tuned Model Continuation: {fine_tuned_continuation}")

# LoRA model continuation (after LoRA fine-tuning)
lora_continuation = generate_continuation(lora_model, tokenizer, prompt)
print(f"LoRA Model Continuation: {lora_continuation}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Pretrained Model Continuation: Inquiry into the application of the United Nations Declaration on the Rights of Indigenous Peoples in Australia.

The application for the Australian Human Rights Act was made by the Department of Justice in the year 2000. The Department has a statutory obligation to ensure that the rights of Aboriginal people are protected. This is a fundamental right that is enshrined in Article 1 of Australia's Declaration of Rights. It is also a right of fundamental importance to the right to which the Commonwealth has to protect. In the case


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Fully Fine-Tuned Model Continuation: Inquiry into the application of the United Nations Declaration on the Rights of Indigenous Peoples in Australia (UNRIP)

The United States of America (USA) (United States) is a United Kingdom of Australia. The United Nation of States is the world's largest and most powerful nation.
. United of United states of america.com
 (www.us.org.au.gov.at.AU) The U.S.A. is an independent nation of
LoRA Model Continuation: Inquiry into the application of the United Nations Declaration on the Rights of Indigenous Peoples in Australia.

The United States of America is a member of a group of nations that are members of an international community. The United Kingdom is the only country in the world that has a treaty with the UN. In the US, the Declaration of Rights is an internationally recognized right. It is also has the right to a free and fair trial. This is why the U.S. has an obligation to


#Applied Parameter-Efficient Fine-Tuning (LoRA):

LoRA Fine-Tuning was applied to the model, and only the added LoRA parameters were trained while keeping the base model frozen.

Trainable Parameters for LoRA: 294,912, which shows that LoRA has fewer trainable parameters compared to the full fine-tuning method, making it more resource-efficient.

#Comparing Resource Use (Trainable Parameters, Training Time):

LoRA Training Time: 0.8260 seconds, which is considerably faster than full fine-tuning because fewer parameters are being trained.

Comparison: The LoRA model is computationally more efficient, and this demonstrates the effectiveness of parameter-efficient fine-tuning.


#Conclusion

In this task, we explored fine-tuning models on domain-specific text using both full fine-tuning and parameter-efficient fine-tuning (LoRA). The fully fine-tuned model demonstrated improved performance on the UNDRIP report, as shown by the drop in perplexity, but also introduced some noise in the generated continuation. LoRA was successfully applied for parameter-efficient fine-tuning, training only a subset of model parameters, which resulted in faster training times and lower resource usage without significant degradation in performance. The generated continuations from both models showed that LoRA captured the key context similar to full fine-tuning, demonstrating the trade-off between efficiency and accuracy. The task successfully demonstrated how LoRA can be a viable alternative for large-scale model fine-tuning, making it a more computationally efficient approach while maintaining high-quality results.